In [17]:
# Load the data

import json
import os

CASES_JSON_PATH = os.environ.get('CASES_JSON_PATH', os.path.join('..', 'cases.json'))

with open(CASES_JSON_PATH) as cases_json_file:
    cases_raw = json.load(cases_json_file)

In [18]:
# Count documents, audio and video
def add_related_counts(case):
    with_related = case.copy()
    for k in ('documents', 'subjects', 'media'):
        del with_related[k]
        with_related[k + '_count'] = len(case[k])
        
    video_count = 0
    audio_count = 0
    for media_item in case['media']:
        if 'vimeo' in media_item['url']:
            video_count += 1
        elif 'soundcloud' in media_item['url']:
            audio_count += 1
            
    with_related['audio_count'] = audio_count
    with_related['video_count'] = video_count
    with_related['attachment_count'] = with_related['documents_count'] + with_related['media_count']
    
    return with_related

cases_with_related_counts = [add_related_counts(c) for c in cases_raw]

In [19]:
# Load everything into an analysis table
import agate

column_types = {
    'log_number': agate.Text(),
}

cases_table = agate.Table.from_object(cases_with_related_counts,
    column_types=column_types)
cases_table.print_table()

|--------------+-----------------+-------------+----------------------+----------------------+------------+------|
|  media_count | documents_count | audio_count | incident_type        |            posted_on | log_number | ...  |
|--------------+-----------------+-------------+----------------------+----------------------+------------+------|
|           18 |               6 |          15 | Firearm Discharge    | 2016-06-03 09:01:... | 1079080    | ...  |
|           11 |               7 |          11 | Firearm Discharge    | 2016-06-03 01:59:... | 1052816    | ...  |
|           19 |              14 |          16 | Firearm Discharge    | 2016-06-03 01:57:... | 1052578    | ...  |
|            3 |               7 |           2 | Firearm Discharge    | 2016-06-03 01:56:... | 1045950    | ...  |
|            9 |               6 |           9 | Firearm Discharge    | 2016-06-03 01:54:... | 1076204    | ...  |
|            2 |              18 |           1 | Firearm Discharge    | 2016-06-

In [47]:
# How many cases are there?
total_cases = len(cases_table.rows)
print("There are {0} total cases".format(total_cases))

There are 101 total cases


In [20]:
# What's the earliest case, the latest?
cases_by_incident_date = cases_table.select(['log_number', 'incident_datetime']).order_by('incident_datetime')
first_case = cases_by_incident_date.rows[0]
last_case = cases_by_incident_date.rows[-1]
print("The first case is {0}, on {1}".format(first_case['log_number'], first_case['incident_datetime']))
print("The last case is {0}, on {1}".format(last_case['log_number'], last_case['incident_datetime']))

The first case is 1042469, on 2011-01-04 21:35:00
The last case is 1079728, on 2016-05-19 21:30:00


In [34]:
# What's the breakdown by year
 
cases_with_year = cases_table.compute([
    ('incident_year', agate.Formula(agate.Number(), lambda row: row['incident_datetime'].year))
])
cases_with_year.pivot('incident_year').print_bars('incident_year', width=80)

incident_year Count
2016              8 ▓░░░░░░░░░                                                  
2012              9 ▓░░░░░░░░░░░                                                
2011              6 ▓░░░░░░░                                                    
2015             44 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░       
2014             24 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░                               
2013             10 ▓░░░░░░░░░░░░                                               
                    +--------------+--------------+-------------+--------------+
                    0.0          12.5           25.0          37.5          50.0


In [23]:
# Which case has the most attachments?
cases_by_attachment_count = cases_table.order_by('attachment_count', reverse=True)
most_attachments = cases_by_attachment_count.rows[0]
print("The case {0} had the most attached documents and media: {1}".format(
    most_attachments['log_number'], most_attachments['attachment_count']))

cases_table.bins('attachment_count').print_bars('attachment_count', width=80)


The case 1053667 had the most attached documents and media: 103
attachment_count Count
[0 - 20)            72 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░      
[20 - 40)           24 ▓░░░░░░░░░░░░░░░░░                                       
[40 - 60)            3 ▓░░                                                      
[80 - 100)           1 ▓░                                                       
[100 - 120)          1 ▓░                                                       
                       +-------------+-------------+-------------+-------------+
                       0            20            40            60            80


In [26]:
# Which case had the most documents?
cases_table.bins('documents_count').print_bars('documents_count', width=80)
cases_by_documents_count = cases_table.order_by('documents_count', reverse=True)
most_documents = cases_by_documents_count.rows[0]
print("The case {0} had the most documents: {1}".format(
    most_documents['log_number'], most_documents['documents_count']))

documents_count Count
[0 - 3)            21 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                           
[3 - 6)            26 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                    
[6 - 9)            32 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░           
[9 - 12)           10 ▓░░░░░░░░░░░░░░                                           
[12 - 15)           8 ▓░░░░░░░░░░░                                              
[15 - 18)           1 ▓░                                                        
[18 - 21)           1 ▓░                                                        
[24 - 27)           1 ▓░                                                        
[27 - 30]           1 ▓░                                                        
                      +-------------+-------------+--------------+-------------+
                      0            10            20             30            40
The case 1053667 had the most documents: 30


In [27]:
# Which case had the most media?
cases_table.bins('media_count').print_bars('media_count', width=80)
cases_by_media_count = cases_table.order_by('media_count', reverse=True)
most_media = cases_by_media_count.rows[0]
print("The case {0} had the most media (audio & video): {1}".format(
    most_media['log_number'], most_media['media_count']))

media_count Count
[0 - 10)       68 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
[10 - 20)      21 ▓░░░░░░░░░░░░░░░░░░                                           
[20 - 30)       6 ▓░░░░░                                                        
[30 - 40)       3 ▓░░░                                                          
[40 - 50)       1 ▓░                                                            
[70 - 80)       1 ▓░                                                            
[90 - 100]      1 ▓░                                                            
                  +--------------+--------------+---------------+--------------+
                  0.0          17.5           35.0            52.5          70.0
The case 1078616 had the most media (audio & video): 91


In [29]:
# Which case had the most audio?
cases_table.bins('audio_count').print_bars('audio_count', width=80)
cases_by_audio_count = cases_table.order_by('audio_count', reverse=True)
most_audio = cases_by_audio_count.rows[0]
print("The case {0} had the most audio: {1}".format(
    most_audio['log_number'], most_audio['audio_count']))

audio_count Count
[0 - 4)        49 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 
[4 - 8)        20 ▓░░░░░░░░░░░░░░░░░░░░░░░░                                     
[8 - 12)       12 ▓░░░░░░░░░░░░░░░                                              
[12 - 16)       9 ▓░░░░░░░░░░░                                                  
[16 - 20)       6 ▓░░░░░░░                                                      
[20 - 24)       1 ▓░                                                            
[28 - 32)       3 ▓░░░░                                                         
[32 - 36)       1 ▓░                                                            
                  +--------------+--------------+---------------+--------------+
                  0.0          12.5           25.0            37.5          50.0
The case 1060844 had the most audio: 34


In [31]:
# Which case had the most video?
cases_table.bins('video_count').print_bars('video_count', width=80)
cases_by_video_count = cases_table.order_by('video_count', reverse=True)
most_video = cases_by_video_count.rows[0]
print("The case {0} had the most video: {1}".format(
    most_video['log_number'], most_video['video_count']))

video_count Count
[0 - 8)        91 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░     
[8 - 16)        6 ▓░░░░                                                         
[16 - 24)       2 ▓░                                                            
[56 - 64)       1 ▓░                                                            
[72 - 80]       1 ▓░                                                            
                  +--------------+--------------+---------------+--------------+
                  0             25             50              75            100
The case 1078616 had the most video: 73


In [46]:
# How many documents, videos and other shit are there in total?
total_documents = cases_table.aggregate(agate.Sum('documents_count'))
print("There are {0} total documents".format(total_documents))
total_audio = cases_table.aggregate(agate.Sum('audio_count'))
print("There are {0} total audio files".format(total_audio))
total_video = cases_table.aggregate(agate.Sum('video_count'))
print("There are {0} total video files".format(total_video))

There are 621 total documents
There are 599 total audio files
There are 322 total video files


In [37]:
# What's the time between incident date and ipra notification
def time_to_notification(row):
    return (row['ipra_notification_date'] - row['incident_datetime'].date()).days

with_time_to_notification = cases_table.compute([
    ('time_to_notification', agate.Formula(agate.Number(), time_to_notification))
])

with_time_to_notification.bins('time_to_notification').print_bars('time_to_notification', width=80)

by_time_to_notification = with_time_to_notification.order_by('time_to_notification', reverse=True)
longest_time_to_notification = by_time_to_notification.rows[0]
print("The case {0} had the longest time from the incident date to when IPRA was notified: {1} days".format(
    longest_time_to_notification['log_number'], longest_time_to_notification['time_to_notification']))

time_to_notification Count
[-100 - 200)            98 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 
[500 - 800)              2 ▓░                                                   
[800 - 1,100)            1 ▓░                                                   
                           +------------+------------+------------+------------+
                           0           25           50           75          100
The case 1078329 had the longest time from the incident date to when IPRA was notified: 1087 days


In [41]:
# Wat? There are negative notification times.  What's up with that?
with_time_to_notification.pivot('time_to_notification').order_by('time_to_notification').print_bars('time_to_notification', width=80)

with_negative_time_to_notification = with_time_to_notification.where(lambda row: row['time_to_notification'] < 0)\
    .select(['log_number', 'incident_datetime', 'ipra_notification_date', 'time_to_notification'])
print("These cases have negative time to notifications")
with_negative_time_to_notification.print_table()

time_to_notification Count
-61                      1 ▓░                                                   
0                       78 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 
1                        9 ▓░░░░░░                                              
2                        3 ▓░░                                                  
3                        2 ▓░                                                   
4                        1 ▓░                                                   
23                       1 ▓░                                                   
52                       1 ▓░                                                   
155                      1 ▓░                                                   
158                      1 ▓░                                                   
505                      1 ▓░                                                   
668                      1 ▓░                                                   
1

In [42]:
# What's the breakdown by incident type?
cases_table.pivot('incident_type').print_bars('incident_type', width=80)

incident_type              Count
Firearm Discharge             74 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   
Incident in Police Custody    25 ▓░░░░░░░░░░░░░░                                
Other Use Of Force             2 ▓░                                             
                                 +-----------+----------+----------+-----------+
                                 0          20         40         60          80
